# Advanced Dataframes

## Agenda

1. Part I: Creating Dataframes
    - from Lists, Arrays, & Dictionaries
    - from PyDataset
    - from a SQL query
2. Exercises, Part I
3. Part II
    - Indexing and Subsetting
    - Aggregating
    - Merging & Joining
4. Exercises, Part II
5. Part III: Reshaping & Transposing
    - Reshaping
    - Transposing
10. Exercises III

In [9]:
# imports
import pandas as pd
import numpy as np

## Part I: Creating Dataframes

### From Lists

In [10]:
pd.DataFrame([[1,2,3],[4,5,6]])

,0,1,2
0,1,2,3
1,4,5,6


### From Arrays

In [11]:
a = np.array([[1,2,3],[4,5,6]])
pd.DataFrame(a, columns = ['a','b','c'])

,a,b,c
0,1,2,3
1,4,5,6


### From Dictionaries

In [12]:
pd.DataFrame({'a':[1,4],
             'b':[2,5],
             'c':[3,6]
             })

,a,b,c
0,1,2,3
1,4,5,6


### From PyDataset

In [13]:
from pydataset import data

### From SQL

We can use the read_sql method to create a dataframe based on the results of a SQL query. To do this, we need to tell pandas how to connect to the database we are querying. The way we communicate this to pandas is with a specially formatted connection string.

In addition, whenever we want to connect to a database from our python code (other programming languages are similar), we will need a driver, a bit of software that handles the details of the database connection.

1. In order to connect to mysql, we'll install the pymysql driver packages by typing the following in your terminal: `python -m pip install pymysql'
1. **Add env.py to the repo's .gitignore file**
2. create a text file called env.py, and in your text file, write the following (filling in the corresponding info):

```python
host = '<ip_address>'
username = '<your_username>'
password = '<your_password>'
```

4. import the host, username, and password variables from the env.py file. Note that upon import, you use `env`, not `env.py`. **once you assign the string to the variable `url`, DO NOT print the value of url to your notebook. If you do this and push the notebook to github, your username and password will be visible to others.**

In [14]:
from pydataset import data
import pandas as pd
import numpy as np
from env import host, username, password 

# create the url variable (string) with my username, password and host inserted
url = f'mysql+pymysql://{username}:{password}:@{host}/employees'
    

In [15]:
# write the SQL query to gather data, assign to variable 'query'
query = 'SELECT * FROM employees LIMIT 10'

In [16]:
df = pd.read_sql(query, url)
# read the data passing the function the query and the url string

OperationalError: (pymysql.err.OperationalError) (1045, "Access denied for user 'noether_2022'@'rrcs-71-78-74-186.sw.biz.rr.com' (using password: YES)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
# write a query in multiple lines:

In [17]:
query = '''

SELECT
    emp_no,
    first_name,
    last_name,
FROM employees
WHERE gender = 'F'
LIMIT 100;
'''

employees_df = pd.read_sql(query, url)
employees_df.head()

OperationalError: (pymysql.err.OperationalError) (1045, "Access denied for user 'noether_2022'@'rrcs-71-78-74-186.sw.biz.rr.com' (using password: YES)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

## Exercises, Part I

Create a notebook or python script named `advanced_dataframes` to do your work in for these exercises.

1. Run python -m pip install pymysql from your terminal to install the mysql client (any folder is fine)

2. cd into your exercises folder for this module and run echo env.py >> .gitignore

3. Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

4. Use your function to obtain a connection to the employees database.

5. Once you have successfully run a query:
    - Intentionally make a typo in the database url. What kind of error message do you see?
    - Intentionally make an error in your SQL query. What does the error message look like?


6. Read the employees and titles tables into two separate DataFrames.

7. How many rows and columns do you have in each DataFrame? Is that what you expected?

8. Display the summary statistics for each DataFrame.

9. How many unique titles are in the titles DataFrame?

10. What is the oldest date in the to_date column?

11. What is the most recent date in the to_date column?

In [18]:
import os
import env 
def get_db_url(db, env_file = os.path.exists('env.py')):
    '''
    this will return a formatted string containing username, password, host, and database
    for connecting to the mysql server and the database indicated.
    env_file checks to see if the env.py exists in the cwd
    
    '''
    if env_file:
        username, password, host = (env.username, env.password, env.host)
        return f'mysql+pymysql://{username}:{password}@{host}/{db}'
    else:
        return 'you need to a username'
    
url = get_db_url('employees')

## Part II

### Indexing and Subsetting

- `[]`: subset rows using a boolean mask, or subset columns using a list of column names. 
- `.loc`: allows for subsetting rows and columns simultaneously, using the labels/names of rows/columns. 
- `.iloc`: allows for subsetting rows and columns simultaneously, using the index location or position of rows/columns. 

In [19]:
# Create list of values for names column.
import pandas as pd
import numpy as np

np.random.seed(123)

names = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas', 'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']


In [20]:
# Randomly generate arrays of scores for each student for each subject.
# Note that all the values need to have the same length here.

math_grades = np.random.randint(low=60, high=101, size=len(names))
english_grades = np.random.randint(low=60, high=101, size=len(names))
reading_grades = np.random.randint(low=60, high=101, size=len(names))

classroom = np.random.choice(['A','B'], len(names))

In [21]:
# Construct the DataFrame using the above lists and arrays.


df = pd.DataFrame({'names': names,
                    'math': math_grades,
                      'english': english_grades,
                      'reading': reading_grades,
                      'classroom': classroom
                      })

Bracket: `[]`

In [22]:
# select 2 columns from the class
df[['names','classroom']]

,names,classroom
0,Sally,A
1,Jane,B
2,Suzie,A
3,Billy,B
4,Ada,A
5,John,B
6,Thomas,A
7,Marie,A
8,Albert,A
9,Richard,A


In [23]:
# I can pass a boolean Series to the indexing operator as a selector.
bools = df.names.str.startswith('A')

In [24]:
# then subset using brackets with the boolean series

`.loc`

- Select specific rows AND columns by index **label**. 
- The index label can be a number, but it can also be a string label. 
- It is **inclusive.**

`df.loc[row_indexer, column_indexer]`

In [25]:
# Select all the rows and a subset of columns; 
# notice the inclusive behavior of the indexing.
df.loc[:,'math':'reading'].head(5)

,math,english,reading
0,62,85,80
1,88,79,67
2,94,74,95
3,98,96,88
4,77,92,98


In [26]:
# I can use a boolean Series as a selector with .loc, too, 
# but I can choose rows and columns.
df.loc[bools, 'math':'reading']

,math,english,reading
4,77,92,98
8,92,62,87
11,92,62,72


`.iloc`

- Select specific rows and colums by index **position**. 
- It does **not** accept a boolean Series as a selector like .loc does. 
- It takes in integers representing index position
- It is **exclusive**.

`df.iloc[row_indexer, column_indexer]`

In [27]:
# select rows by integer position
# notice the exclusive behavior of the indexing

In [28]:
df.iloc[:4, 2:5]

,english,reading,classroom
0,85,80,A
1,79,67,B
2,74,95,A
3,96,88,B


### Aggregating

`.agg`

- Specify a way to aggregate a series of numerical values. 
- We pass an aggregate function or list of functions to the method that we want applied to a Series.

In [29]:
df.reading.agg('min')

67

What happens if you run it on the entire dataframe, not just a series of a single column? 

In [30]:
# aggregate each column in the dataframe by taking the min value
df[['math','english','reading']].agg(['mean', 'min','max']).round(2)

,math,english,reading
mean,84.83,77.67,86.5
min,62.00,62.00,67.0
max,98.00,99.00,98.0


While on the surface this seems pretty simple, `.agg` is capable of providing more detailed aggregations:

`groupby`

- Creates a grouped object. 
- Then apply an aggregation on that object. 

For example, if we wanted to know the highest math grade from each classroom:

In [31]:
df.groupby('classroom').math.agg(['mean'])

,mean
classroom,
A,82.625
B,89.250


We can use `.agg` here to, to see multiple aggregations:

In [32]:
df.groupby('classroom')[['math','english']].agg(['mean',
                                                    'median',
                                                    'max',
                                                    'min'
                                                    ])

#nested columns

math                english               
             mean median max min    mean median max min
classroom                                              
A          82.625   87.0  94  62   72.75   69.0  92  62
B          89.250   90.0  98  79   87.50   87.5  99  76

We can group by multiple columns as well. 

Let's compute the average reading grade for students who are passing english and those who are not passing english. 


In [33]:
# Create a boolean column named passing_english 
# based on the math column using np.where
df['passing_math'] = np.where(df.math < 70, 'failing', 'passing')
df

,names,math,english,reading,classroom,passing_math
0,Sally,62,85,80,A,failing
1,Jane,88,79,67,B,passing
2,Suzie,94,74,95,A,passing
3,Billy,98,96,88,B,passing
4,Ada,77,92,98,A,passing
5,John,79,76,93,B,passing
6,Thomas,82,64,81,A,passing
7,Marie,93,63,90,A,passing
8,Albert,92,62,87,A,passing
9,Richard,69,80,94,A,failing


In [34]:
# Group by the combination of our new feature, passing_english,
# and the classroom.
# aggregate by calculating average reading grade
grade_groups = df.groupby(['classroom','passing_math']).reading.agg(['mean','count'])

In [35]:
# clean up the column names to make the data more transparent
grade_groups.index
grade_groups.columns = ['avg_reading_grade', 'number_of_students']
grade_groups

avg_reading_grade  number_of_students
classroom passing_math                                       
A         failing               87.000000                   2
          passing               87.166667                   6
B         passing               85.250000                   4

`.transform`

When you want to add an aggregate value, like the average math grades for each classroom, back to the original dataframe. 

In [36]:
df.groupby('classroom').math.agg('mean')

classroom
A    82.625
B    89.250
Name: math, dtype: float64

In [37]:
df.assign(avg_math_by_classroom = df.groupby('classroom').math.transform('mean'))
#transform works to spread it across the dataframe

,names,math,english,reading,classroom,passing_math,avg_math_by_classroom
0,Sally,62,85,80,A,failing,82.625
1,Jane,88,79,67,B,passing,89.250
2,Suzie,94,74,95,A,passing,82.625
3,Billy,98,96,88,B,passing,89.250
4,Ada,77,92,98,A,passing,82.625
5,John,79,76,93,B,passing,89.250
6,Thomas,82,64,81,A,passing,82.625
7,Marie,93,63,90,A,passing,82.625
8,Albert,92,62,87,A,passing,82.625
9,Richard,69,80,94,A,failing,82.625


In [38]:
df

,names,math,english,reading,classroom,passing_math
0,Sally,62,85,80,A,failing
1,Jane,88,79,67,B,passing
2,Suzie,94,74,95,A,passing
3,Billy,98,96,88,B,passing
4,Ada,77,92,98,A,passing
5,John,79,76,93,B,passing
6,Thomas,82,64,81,A,passing
7,Marie,93,63,90,A,passing
8,Albert,92,62,87,A,passing
9,Richard,69,80,94,A,failing


`.describe` 

Can be used with `groupby` also. 

In [39]:
df.groupby('classroom').describe().T

classroom              A          B
math    count   8.000000   4.000000
        mean   82.625000  89.250000
        std    12.281665   7.973916
        min    62.000000  79.000000
        25%    75.000000  85.750000
        50%    87.000000  90.000000
        75%    92.250000  93.500000
        max    94.000000  98.000000
english count   8.000000   4.000000
        mean   72.750000  87.500000
        std    11.817058  11.676187
        min    62.000000  76.000000
        25%    62.750000  78.250000
        50%    69.000000  87.500000
        75%    81.250000  96.750000
        max    92.000000  99.000000
reading count   8.000000   4.000000
        mean   87.125000  85.250000
        std     8.887190  12.392874
        min    72.000000  67.000000
        25%    80.750000  82.750000
        50%    88.500000  90.500000
        75%    94.250000  93.000000
        max    98.000000  93.000000

### Merging & Joining

Ways to combine dataframes together

`pd.concat`

- This function takes in a list or dictionary of Series or DataFrame objects and joins them along a particular axis
- Row-wise: axis=0, means your new data frame will be longer with more rows. 
- Column-wise: axis=1, means your new dataframe will be wider with more columns. 
- When your list contains at least one DataFrame, a DataFrame is returned.
- When concatenating only Series objects row-wise, axis=0, a Series is returned.
- When concatenating Series or DataFrames column-wise, axis=1, a DataFrame is returned.

default parameters: 

`pd.concat(objs_in_list, axis=0, join='outer')`

In [40]:
# concatenate 2 dataframes by passing a list of the 2 dfs, and 
# indicating whether to concatenate along the rows or the columns

In [41]:
df.shape

(12, 6)

In [42]:
df1 = pd.DataFrame({'a': [1,2,3]})
df2 = pd.DataFrame({'a': [98,6,5]})
pd.concat([df1,df2], axis=0)

,a
0,1
1,2
2,3
0,98
1,6
2,5


`.merge`

- Similar to a SQL join.
- first dataframe is the 'left' table. 
- `how`: Type of merge to be performed.
    - `how=left`: use only keys from left frame, similar to a SQL left outer join; preserve key order.
    - `how=right`: use only keys from right frame, similar to a SQL right outer join; preserve key order.
    - `how=outer`: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.
    - `how=inner`: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.


In [43]:
# Create the users DataFrame.
users = pd.DataFrame({
    'id':[1,2,3,4,5,6],
    'name':['bob','joe','sally','adam','jorge', 'mike'],
    'role_id':[1,2,3,3,np.nan,np.nan]
    
    
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jorge,NaN
5,6,mike,NaN


In [44]:
# Create the roles DataFrame
roles = pd.DataFrame({
    'id': [1,2,3,4],
    'name':['admin','author','reviewer','commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


- `left_on` and `right_on`: indicate the columns that are the keys used to merge the dataframes together.

- `indicator`=`True` will create a column indicating whether the merge key appears in the `left_only`, `right_only` or `both` DataFrames.

In [45]:
# Perform an outer join specifying the left and right DataFrame keys.
users.merge(roles, left_on='role_id', right_on = 'id', how='outer',indicator = True)

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jorge,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


- suffix of `_x` to any columns in the left dataframe that are duplicated, 
- suffix of `_y` to any columns in the right dataframe that are duplicated. 
- can clean up column names using **method chaining**.

In [46]:
(users.merge(roles, left_on='role_id', right_on = 'id', how='outer',indicator = True)
             .drop(columns = 'role_id')
            .rename(columns={'id_x':'id',
                            'name_x':'employee',
                             'id_y':'role_id',
                             'name_y':'role'}))

,id,employee,role_id,role,_merge
0,1.0,bob,1.0,admin,both
1,2.0,joe,2.0,author,both
2,3.0,sally,3.0,reviewer,both
3,4.0,adam,3.0,reviewer,both
4,5.0,jorge,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,left_only
6,NaN,NaN,4.0,commenter,right_only


## Part 3: Reshaping and Transposing

1. Reshaping: pd.crosstab, .pivot_table
2. Transposing

### Reshaping

`pd.crosstab`

Count the number of students passing math in each classroom (student grades df):

In [47]:
df

,names,math,english,reading,classroom,passing_math
0,Sally,62,85,80,A,failing
1,Jane,88,79,67,B,passing
2,Suzie,94,74,95,A,passing
3,Billy,98,96,88,B,passing
4,Ada,77,92,98,A,passing
5,John,79,76,93,B,passing
6,Thomas,82,64,81,A,passing
7,Marie,93,63,90,A,passing
8,Albert,92,62,87,A,passing
9,Richard,69,80,94,A,failing


In [50]:
pd.crosstab(df.passing_math,df.classroom)

classroom,A,B
passing_math,,
failing,2,0
passing,6,4


Add subtotals using the `margins` argument:

In [49]:
pd.crosstab(df.passing_math, df.classroom,
            margins=True)

classroom,A,B,All
passing_math,,,
failing,2,0,2
passing,6,4,10
All,8,4,12


View percentages instead of actual values using the `normalize` argument. 

In [52]:
pd.crosstab(df.passing_math, df.classroom,
            margins=True, normalize ='columns').round(2)

classroom,A,B,All
passing_math,,,
failing,0.25,0.0,0.17
passing,0.75,1.0,0.83


`.pivot_table`

Like an excel pivot table

Supply 4 pieces of info to arguments: 
- which values will make up the rows, the `index`
- which values will make up the columns
- the values we are aggregating
- an aggregation method (`aggfunc`) (default is `mean`). 

Example: Calculate the average math grade for the combination of `classroom` and `passing_math` status. 

In [58]:
df.pivot_table(index='classroom', 
               columns= 'passing_math', 
               values = 'math', 
               aggfunc= ['mean','sum'])

mean                sum        
passing_math failing    passing failing passing
classroom                                      
A               65.5  88.333333   131.0   530.0
B                NaN  89.250000     NaN   357.0

Create a new dataframe: orders at a restaurant. 

In [59]:
n = 40

orders = pd.DataFrame({
    'drink': np.random.choice(['Tea', 'Water', 'Water'], n),
    'meal': np.random.choice(['Curry', 'Yakisoba Noodle', 'Pad Thai'], n),
})

orders.sample(10)


,drink,meal
0,Tea,Pad Thai
38,Water,Curry
11,Tea,Curry
4,Tea,Curry
16,Water,Curry
2,Tea,Yakisoba Noodle
22,Water,Yakisoba Noodle
5,Water,Pad Thai
19,Tea,Pad Thai
15,Water,Yakisoba Noodle


`.map`

Use a dictionary to calculate the total price for an order. Then save the calculations to a new column named `bill`. 

In [60]:


# Create a dictionary of prices for drinks and meals.

prices = {
    'Yakisoba Noodle': 9,
    'Curry': 11,
    'Pad Thai': 10,
    'Tea': 2,
    'Water': 0,
}


In [64]:
# match the values in the drink and meal columns with the values 
# in the prices and perform the specificied calculations. 
# save to a new column, bill

orders['bill'] = orders.drink.map(prices) + orders.meal.map(prices)
orders.head()

,drink,meal,bill
0,Tea,Pad Thai,12
1,Water,Pad Thai,10
2,Tea,Yakisoba Noodle,11
3,Water,Pad Thai,10
4,Tea,Curry,13


In [65]:
# use crosstab to look at how many orders have each combination of
# meal and drink
pd.crosstab(orders.drink, orders. meal)

meal,Curry,Pad Thai,Yakisoba Noodle
drink,,,
Tea,3,4,5
Water,9,10,9


In [66]:
# find the average bill for each combination using pivot_table
orders.pivot_table(index='drink', 
                   columns='meal', 
                   values='bill',
                    aggfunc = 'mean'
                  )

meal,Curry,Pad Thai,Yakisoba Noodle
drink,,,
Tea,13,12,11
Water,11,10,9


In [100]:
# find the average bill for each combination using groupby

Transposing

Swapping rows and columns

In [69]:
orders.groupby(['drink','meal'])['bill'].agg('mean')

drink  meal           
Tea    Curry              13.0
       Pad Thai           12.0
       Yakisoba Noodle    11.0
Water  Curry              11.0
       Pad Thai           10.0
       Yakisoba Noodle     9.0
Name: bill, dtype: float64